In [1]:
import pandas as pd

In [2]:
def load_prices ():
    prices_df = pd.read_csv('plan_yahoo/historical_prices_final.csv')
    prices_df.drop(columns='Unnamed: 0', inplace=True)
    prices_df['timestamps'] = pd.to_datetime(prices_df['timestamps']).dt.normalize()
    return prices_df

In [ ]:
def load_average_return ():
    df = pd.read_csv('plan_kaggle/sp500_stocks.csv')
    sp = pd.read_csv('plan_kaggle/sp500_index.csv')

    # Cleaning columns with snake_case 
    df.columns = [col.lower().replace(" ", "_")for col in df.columns] 
    sp.columns = [col.lower().replace(" ", "_")for col in sp.columns]

    # Delete NaN.
    df.dropna(how='any', inplace=True)
    df.drop(columns=['high', 'low', 'open','close','volume'], inplace=True)

    # Change to datetime from stocks. 
    df['date'] = pd.to_datetime(df['date']) 
    sp['date'] = pd.to_datetime(sp['date'])

    sp['symbol'] = "SP500"
    
    sp.rename(columns={'s&p500':'price',}, inplace=True)
    df.rename(columns={'adj_close':'price',}, inplace=True)

    cols = ['date', 'symbol', 'price']
    sp = sp[cols]

    final_df = pd.concat([df, sp], axis=0)

    final_df['year'] = final_df['date'].dt.year
    final_df['month'] = final_df['date'].dt.month
    final_df['day'] = final_df['date'].dt.day

    final_df.drop(final_df[(final_df['year'] >= 2010) & (final_df['year'] <= 2013)].index, inplace=True)

    final_df.drop(columns=['date'], inplace=True)

    filtered_data = final_df[(final_df['year'] >= 2014) & (final_df['year'] <= 2024)]

    # Get the first and last adjusted close prices for each symbol
    first_last_prices = filtered_data.groupby('symbol').agg(
        first_price=('price', 'first'),
        last_price=('price', 'last')).reset_index()

    # Calculate the annual return from the first and last prices
    first_last_prices['average_return'] = (first_last_prices['last_price'] / first_last_prices['first_price']) - 1
    return first_last_prices

In [2]:
df = pd.read_csv('plan_kaggle/annual_returns.csv')

In [3]:
df

,Unnamed: 0,symbol,year,annual_return
0,4,A,2014,0.0254
1,5,A,2015,0.0418
2,6,A,2016,0.1319
3,7,A,2017,0.4535
4,8,A,2018,0.0072
...,...,...,...,...
5378,7181,ZTS,2020,0.2409
5379,7182,ZTS,2021,0.4999
5380,7183,ZTS,2022,-0.3692
5381,7184,ZTS,2023,0.3563


In [4]:
# Multiply the 'annual_return' column by 100
df["annual_return"] = df["annual_return"] * 100

In [5]:
df

,Unnamed: 0,symbol,year,annual_return
0,4,A,2014,2.54
1,5,A,2015,4.18
2,6,A,2016,13.19
3,7,A,2017,45.35
4,8,A,2018,0.72
...,...,...,...,...
5378,7181,ZTS,2020,24.09
5379,7182,ZTS,2021,49.99
5380,7183,ZTS,2022,-36.92
5381,7184,ZTS,2023,35.63


In [6]:
df[df['symbol'] == "AAPL"]

,Unnamed: 0,symbol,year,annual_return
11,19,AAPL,2014,42.63
12,20,AAPL,2015,-2.08
13,21,AAPL,2016,12.38
14,22,AAPL,2017,48.04
15,23,AAPL,2018,-7.05
16,24,AAPL,2019,88.74
17,25,AAPL,2020,78.24
18,26,AAPL,2021,38.06
19,27,AAPL,2022,-28.20
20,28,AAPL,2023,54.80
